In [ ]:
# %pip install --upgrade pip
# %pip install ollama chromadb

In [ ]:
import ollama, chromadb, pytesseract, datetime, re, json
from pymongo import MongoClient

# import pandas_access as mdb
# import pandas as pd

from IPython.display import Image, display
from llama_index.core import SimpleDirectoryReader
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.readers.web import SimpleWebPageReader

from dotenv import dotenv_values

In [ ]:
db = chromadb.Client()
config = {
    **dotenv_values(".env"),  # load shared development variables
    # **os.environ,  # override loaded values with environment variables
}
embeded_model = config['embeded_model']
chatllm_model = config['chatllm_model']
# tools_model = config['tools_model']
chatllm_model = 'phi3'
tools_model = 'toolla'

In [ ]:
tools = open('./datasets/tools.json')
func_calls = json.load(tools)
func_calls

In [ ]:
default_tool_prompt = """
Base on this query "{question}".
You are a helpful assistant that tells the user which tools to use and which inputs to give to the tools.  The list of descriptions of available tools in JSON format is 
{tool_list}.  
The user will ask a task to be performed and you will respond with the proper tool to use.  If a task requires multiple steps you will return the tool for the first step and the user will respond with an result from the tool used. If no tool is needed, simply answer the question without it and just say the word "no_tools". You will return tool to be used only text inside as JSON format with following structure 
```json
{{ 
    "tool": "<name>", 
    "inputs": {{ 
        "<key>": "<value>"
    }} 
}}
```
"""

In [ ]:
collection = db.get_or_create_collection(name='toolla2')

In [ ]:
# store each document in a vector embedding database
def embedded(doc_splits=[]):
    text_representation = []
    for data in doc_splits:
        text_representation.append(json.dumps(data))
    for i, d in enumerate(text_representation):
        if type(d) is not str:
            d = d.page_content

        response = ollama.embeddings(model=tools_model, prompt=d)
        embedding = response["embedding"]
        collection.upsert(
            ids=[str(i)],
            embeddings=[embedding],
            documents=[d]
        )

def get_tools(prompt=None):
    response = ollama.embeddings(model=tools_model, prompt=prompt)
    print(response)
    results = collection.query(query_embeddings=[response["embedding"]], n_results=3)
    return results


In [ ]:
embedded(doc_splits=func_calls)

In [ ]:
prompt="Give a list of songs and albums"
res = get_tools(prompt=prompt)
# data = json.loads(res['documents'][0][0])
chat = ollama.generate(model='llama3.1', prompt=default_tool_prompt.format(tool_list=res['documents'], question=prompt))
chat['response']

In [ ]:
from langchain_community.llms import OpenAI
from langchain_core.tools import tool

llm = OpenAI(model_name='phi3', openai_api_base="http://127.0.0.1:11434/v1", openai_api_key='ollama')

In [ ]:
res = llm.invoke('Give a list of songs and albums')

In [ ]:
tools

In [ ]:
# results = collection.query(
#     query_texts=["set alarm to 12:00:00"], # Chroma will embed this for you
#     n_results=2 # how many results to return
# )

In [ ]:
res

In [ ]:
res['documents'][0][0]

In [ ]:
%pip install langchain-community

In [ ]:
from langchain_community.utilities import SearxSearchWrapper
from langchain.agents import load_tools
import pprint

In [ ]:
tools = load_tools(["searx-search-results-json"], searx_host="http://localhost:8888", num_results=5)

In [ ]:

search = SearxSearchWrapper(searx_host="https://psarai.com")

In [ ]:
results = search.results(
    "Large Language Model prompt",
    num_results=5,
    engine="google",
    safe_mode=True,
    # language="English",
    # sort_by="relevance",
    # date_range="past_year",
    # categories="science",
    # time_range="year",
)
pprint.pp(results)

In [ ]:
import os
from langchain_community.llms import Ollama
from dotenv import load_dotenv
from langchain_community.embeddings import OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import create_retrieval_chain
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain

In [ ]:
llm = Ollama(model="llama3.1", base_url="http://127.0.0.1:11434")

In [ ]:
embed_model = OllamaEmbeddings(
    model="nomic-embed-text",
    base_url='http://127.0.0.1:11434'
)

In [ ]:
text = """
In the lush canopy of a tropical rainforest, two mischievous monkeys, Coco and Mango, swung from branch to branch, their playful antics echoing through the trees. They were inseparable companions, sharing everything from juicy fruits to secret hideouts high above the forest floor. One day, while exploring a new part of the forest, Coco stumbled upon a beautiful orchid hidden among the foliage. Entranced by its delicate petals, Coco plucked it and presented it to Mango with a wide grin. Overwhelmed by Coco's gesture of friendship, Mango hugged Coco tightly, cherishing the bond they shared. From that day on, Coco and Mango ventured through the forest together, their friendship growing stronger with each passing adventure. As they watched the sun dip below the horizon, casting a golden glow over the treetops, they knew that no matter what challenges lay ahead, they would always have each other, and their hearts brimmed with joy.
"""

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
chunks = text_splitter.split_text(text)
chunks

In [ ]:
vector_store = Chroma.from_texts(chunks, embed_model)

In [ ]:
retriever = vector_store.as_retriever()
retriever

In [ ]:
chain = create_retrieval_chain(combine_docs_chain=llm,retriever=retriever)

In [ ]:
# %pip install langchainhub

In [ ]:
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [ ]:
combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)

In [ ]:
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [ ]:
response = retrieval_chain.invoke({"input": "Tell me name of monkeys and where do they live"}).tool_calls
response

In [ ]:
print(response['answer'])

New tools

In [ ]:
%pip install -qU langchain-openai

In [17]:

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.pydantic_v1 import BaseModel, Field

In [18]:
llms = ChatOpenAI(model="phi3", base_url="http://127.0.0.1:11434/v1", api_key='ollama')

In [19]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

In [4]:
tools

[StructuredTool(name='add', description='Adds a and b.', args_schema=<class 'pydantic.main.addSchema'>, func=<function add at 0x70bbf0541990>),
 StructuredTool(name='multiply', description='Multiplies a and b.', args_schema=<class 'pydantic.main.multiplySchema'>, func=<function multiply at 0x70bbf0541a20>)]

In [20]:
import json
tools = open('./datasets/tools.json')
func_calls = json.load(tools)
func_calls

[{'type': 'function',
  'function': {'name': 'add_alarm',
   'description': 'Add an alarm to the active timers. Displayed as an alarm for the given time.',
   'parameters': {'type': 'object',
    'properties': {'time': {'type': 'string',
      'description': "The exact time when the timer should go off, in the format 'YYYY-MM-DD HH:MM:SS'."},
     'title': {'type': 'string',
      'description': 'Optional title of the timer.'}},
    'required': ['time']}}},
 {'type': 'function',
  'function': {'name': 'add_countdown',
   'description': 'Add a countdown timer to the active timers. Displayed as counting down to zero.',
   'parameters': {'type': 'object',
    'properties': {'duration': {'type': 'string',
      'description': 'A duration in ISO 8601 format.'},
     'title': {'type': 'string',
      'description': 'Optional title of the timer.'}},
    'required': ['duration']}}},
 {'type': 'function',
  'function': {'name': 'delete_timer',
   'description': 'Cancel one of the active timers'

In [21]:
llm_with_tools = llms.bind_tools(func_calls)

In [22]:
query = "Give me a song of Justin Bieber"

res = llm_with_tools.invoke(query).tool_calls
res

[]

In [ ]:
# chain = llm_with_tools | PydanticToolsParser(tools=tools)
llm_with_tools.invoke(query).tool_calls